In [ ]:
import numpy as np
import scipy as sp
from sympy import Symbol, Poly
import matplotlib.pyplot as plt
import resource
from IPython.display import display

In [ ]:
import sys
sys.path.append("..")

from ggqpy import construct_Chebyshev_quadratures
from ggqpy.utils import Interval, FunctionFamily
from ggqpy.testproblems import example_problem, gen_poly_and_sing
from ggqpy.discretize import Discretizer, compress_sequence_of_functions, construct_A_matrix
from ggqpy.optimize import QuadOptimizer
from ggqpy.visualize import plot_points, visualise_diagonal_dropoff

In [ ]:
eps_disc = 1e-10
eps_comp = 1e2*eps_disc
I = Interval(-1,1)
seed = 0

x = Symbol("x", real = True)
F = FunctionFamily(I, [1 + 1e-100*x] + [x**k for k in range(1,30)])
ex_f, ex_f_expr = F.generate_example_function()

In [ ]:
number_of_points = 100
disc = Discretizer(eps_disc, min_length=1e-4, interpolation_degree=number_of_points) 
x_disc, w_disc, endpoints, intervals = disc.adaptive_discretization(F)

In [ ]:
U_disc, rank = compress_sequence_of_functions(F.functions,(x_disc,), w_disc, eps_comp)
u_list = disc.interpolate_piecewise_legendre(U_disc)

In [ ]:
(x_cheb,),w_cheb = construct_Chebyshev_quadratures((x_disc,),w_disc,U_disc)
plt.stem(x_cheb,w_cheb)

x_gl,w_gl = np.polynomial.legendre.leggauss(10)
# plt.stem(x_gl,w_gl,markerfmt="r")
x_cheb**2@w_cheb

In [ ]:
r = U_disc.T@w_disc
eps_quad = 0.1
opt = QuadOptimizer(u_list,r)
x,w = opt.reduce_quadrature(x_cheb,w_cheb,eps_quad)

In [ ]:
x_gl,w_gl = np.polynomial.legendre.leggauss(len(x))
plt.stem(x_gl,w_gl,markerfmt="r")
plt.stem(x,w)
print(x_gl-sorted(x))
w.sum()